In [5]:
import sys
sys.path.append('..')
from config import POSTGRES_UTEA

# conexion postgress
import psycopg2

import os
import requests
import time
import random

In [12]:
def numero_random():
    return random.randint(3, 30)

In [7]:
def sendMessage(para, mensaje):
    """
    Envía un mensaje a través de la API de UTEA-CHAT con manejo de errores.
    
    - `para`: Número de teléfono (sin el código de país).
    - `mensaje`: Contenido del mensaje a enviar.
    """
    
    url = "http://localhost:8080/message/sendText/UTEA - DRON"
    payload = {
        "number": '591' + str(para),  # Agrega el código de país (Bolivia: 591)
        "text": mensaje,
    }
    headers = {
        "apikey": "ABA8B719884D-485F-B56A-A92A1C2CCC4C",
        "Content-Type": "application/json"
    }

    try:
        response = requests.request("POST", url, json=payload, headers=headers, timeout=10)

        # Verifica si la respuesta fue exitosa (código 200-299)
        response.raise_for_status()
        print(f"✅ Mensaje enviado correctamente a {para}: {mensaje}")
        return response.json()  # Retorna la respuesta en formato JSON

    except requests.ConnectionError:
        print(f"❌ Error de conexión al intentar enviar mensaje a {para}.")

    except requests.Timeout:
        print(f"⏳ Tiempo de espera agotado al intentar enviar mensaje a {para}.")

    except requests.HTTPError as e:
        print(f"⚠️ Error HTTP {response.status_code} al enviar mensaje a {para}: {response.text}")

    except requests.RequestException as e:
        print(f"❌ Error inesperado al enviar mensaje a {para}: {e}")

    return None  # Retorna None si ocurrió un error


In [8]:
def obtener_conexion():
    return psycopg2.connect(
        host=POSTGRES_UTEA["HOST"],
        port=POSTGRES_UTEA["PORT"],
        database=POSTGRES_UTEA["DATABASE"],
        user=POSTGRES_UTEA["USER"],
        password=POSTGRES_UTEA["PASSWORD"]
    )

In [9]:
def obtener_mensajes_no_enviados():
    try:
        conexion = obtener_conexion()
        cursor = conexion.cursor()
        
        cursor.execute("""
            SELECT id, cod_canero, nombre_canero, numero_cel, mensaje
            FROM msj_whatsapp
            WHERE enviado = FALSE
        """)
        
        mensajes = cursor.fetchall()
        
        cursor.close()
        conexion.close()
        
        return mensajes

    except Exception as e:
        print("Error al obtener mensajes no enviados:", e)
        return []

In [10]:
def marcar_como_enviado(id_mensaje):
    try:
        conexion = obtener_conexion()
        cursor = conexion.cursor()

        cursor.execute("""
            UPDATE msj_whatsapp
            SET enviado = TRUE,
                fecha_enviado = CURRENT_TIMESTAMP
            WHERE id = %s
        """, (id_mensaje,))

        conexion.commit()
        cursor.close()
        conexion.close()
        print(f"Mensaje con ID {id_mensaje} marcado como enviado.")
    except Exception as e:
        print(f"Error al actualizar mensaje {id_mensaje}: {e}")

In [11]:
def enviar_mensajes():
    mjs_pendientes = obtener_mensajes_no_enviados()
    if len(mjs_pendientes) == 0:
        print(f"No existen mensajes nuevos para enviar...!!")
    for i in mjs_pendientes:
        id = i[0]
        cod_canero = i[1]
        nombre_canero = i[2]
        numero_cel = i[3]
        mensaje = i[4]
        if (sendMessage(numero_cel, mensaje)):
            marcar_como_enviado(id)
            time.sleep(5)

In [ ]:
while True:
    enviar_mensajes()
    time.sleep(numero_random()) 

✅ Mensaje enviado correctamente a 76344582: Buenos días, Sr(a). AGUILERA TARADELLES NELSON MARIANO 👋🏼

Le saludamos desde el Equipo de Drones del CITTCA para informarle que estamos recibiendo solicitudes para el servicio de pulverización.

Si desea reservar, puede responder a este mensaje con un formato como el siguiente:
👉 Deseo reservar 70 has para la 2da quincena de junio.

Esto nos permitirá incluirlo en la programación y darle prioridad en las fechas solicitadas.

Agradecemos mucho su confianza y quedamos atentos a su confirmación.
¡Saludos cordiales! 🌱✈️
Mensaje con ID 8 marcado como enviado.
✅ Mensaje enviado correctamente a 77054999: Buenos días, Sr(a). AGUILERA TARADELLES NELSON MARIANO 👋🏼

Le saludamos desde el Equipo de Drones del CITTCA para informarle que estamos recibiendo solicitudes para el servicio de pulverización.

Si desea reservar, puede responder a este mensaje con un formato como el siguiente:
👉 Deseo reservar 70 has para la 2da quincena de junio.

Esto nos permi